In [7]:
# nto25_notebook.ipynb

# %% [markdown]
# # NTO25 ML Competition Baseline
# Полный пайплайн для соревнования по машинному обучению

# %%
# Установка зависимостей (раскомментируйте при необходимости)
# !pip install lightgbm pandas numpy scikit-learn tqdm joblib pyarrow

# %%
import sys
import os
import warnings
warnings.filterwarnings('ignore')

import time
from pathlib import Path
from typing import Any, List, Dict
from datetime import datetime

import joblib
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

# Проверяем наличие LightGBM
try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
    print("✅ LightGBM успешно импортирован")
except Exception as e:
    print(f"⚠️  LightGBM не найден: {e}")
    print("Используем альтернативную модель (RandomForest)...")
    LGB_AVAILABLE = False
    # Импортируем RandomForest как альтернативу
    from sklearn.ensemble import RandomForestRegressor
    # Создаем псевдоним для совместимости
    class LGBMRegressorWrapper:
        def __init__(self, **kwargs):
            # Преобразуем параметры LightGBM в параметры RandomForest
            n_estimators = kwargs.get('n_estimators', 100)
            max_depth = kwargs.get('num_leaves', 31)
            min_samples_split = kwargs.get('min_child_samples', 20)
            
            self.model = RandomForestRegressor(
                n_estimators=min(n_estimators, 100),  # Ограничиваем для скорости
                max_depth=min(max_depth, 20),
                min_samples_split=min_samples_split,
                random_state=kwargs.get('seed', 42),
                n_jobs=-1,
                verbose=0
            )
        
        def fit(self, X, y, eval_set=None, eval_metric=None, callbacks=None, verbose=0):
            # Игнорируем eval_set для RandomForest
            self.model.fit(X, y)
        
        def predict(self, X):
            return self.model.predict(X)
        
        @property
        def feature_importances_(self):
            return self.model.feature_importances_
    
    # Создаем псевдонимы
    lgb = type('lgb', (), {
        'LGBMRegressor': LGBMRegressorWrapper,
        'early_stopping': lambda **kwargs: None
    })()

# Для ALS (неявная обратная связь)
try:
    import implicit
    from implicit.als import AlternatingLeastSquares
    from scipy.sparse import coo_matrix, csr_matrix
    ALS_AVAILABLE = True
    print("✅ Implicit (ALS) успешно импортирован")
except ImportError:
    print("⚠️  Библиотека implicit не установлена. ALS фичи будут отключены.")
    print("Установите: pip install implicit")
    ALS_AVAILABLE = False
except Exception as e:
    print(f"⚠️  Ошибка импорта implicit: {e}")
    ALS_AVAILABLE = False

# %%
# =============================================================================
# КОНСТАНТЫ
# =============================================================================

# --- ФАЙЛЫ ---
TRAIN_FILENAME = "/home/evstigneva/nto252/train.csv"
TEST_FILENAME = "/home/evstigneva/nto252/test.csv" 
USER_DATA_FILENAME = "/home/evstigneva/nto252/users.csv"
BOOK_DATA_FILENAME = "/home/evstigneva/nto252/books.csv"
BOOK_GENRES_FILENAME = "/home/evstigneva/nto252/book_genres.csv"
GENRES_FILENAME = "/home/evstigneva/nto252/genres.csv"
BOOK_DESCRIPTIONS_FILENAME = "/home/evstigneva/nto252/book_descriptions.csv"
SUBMISSION_FILENAME = "submission_with_toread_features.csv"
TFIDF_VECTORIZER_FILENAME = "tfidf_vectorizer.pkl"
PROCESSED_DATA_FILENAME = "processed_features_with_toread.parquet"
ALS_MODEL_FILENAME = "als_model.pkl"
USER_ITEM_MATRIX_FILENAME = "user_item_matrix.pkl"

# --- НАЗВАНИЯ КОЛОНОК ---
COL_USER_ID = "user_id"
COL_BOOK_ID = "book_id"
COL_TARGET = "rating"
COL_SOURCE = "source"
COL_PREDICTION = "rating_predict"
COL_HAS_READ = "has_read"
COL_TIMESTAMP = "timestamp"

# Фичевые колонки (базовые)
F_USER_MEAN_RATING = "user_mean_rating"
F_USER_RATINGS_COUNT = "user_ratings_count"
F_BOOK_MEAN_RATING = "book_mean_rating"
F_BOOK_RATINGS_COUNT = "book_ratings_count"
F_AUTHOR_MEAN_RATING = "author_mean_rating"
F_BOOK_GENRES_COUNT = "book_genres_count"

# Новые фичи из has_read=0 (СТРАТЕГИЯ 2)
F_USER_TOREAD_COUNT = "user_toread_count"
F_USER_READ_TOREAD_RATIO = "user_read_toread_ratio"
F_BOOK_TOREAD_COUNT = "book_toread_count"
F_BOOK_READ_TOREAD_RATIO = "book_read_toread_ratio"
F_IS_IN_TOREAD_LIST = "is_in_toread_list"
F_DAYS_SINCE_LAST_TOREAD = "days_since_last_toread"
F_USER_TOREAD_ACTIVITY = "user_toread_activity"
F_BOOK_TOREAD_POPULARITY = "book_toread_popularity"

# ALS фичи
F_ALS_USER_SCORE = "als_user_score"
F_ALS_BOOK_SCORE = "als_book_score"
F_ALS_DOT_PRODUCT = "als_dot_product"

# Метаданные из сырых данных
COL_GENDER = "gender"
COL_AGE = "age"
COL_AUTHOR_ID = "author_id"
COL_PUBLICATION_YEAR = "publication_year"
COL_LANGUAGE = "language"
COL_PUBLISHER = "publisher"
COL_AVG_RATING = "avg_rating"
COL_GENRE_ID = "genre_id"
COL_DESCRIPTION = "description"

# --- ЗНАЧЕНИЯ ---
VAL_SOURCE_TRAIN = "train"
VAL_SOURCE_TEST = "test"

# --- МАГИЧЕСКИЕ ЧИСЛА ---
MISSING_CAT_VALUE = "-1"
MISSING_NUM_VALUE = -1
PREDICTION_MIN_VALUE = 0
PREDICTION_MAX_VALUE = 10
ALS_FACTORS = 32
TOP_N_GENRES = 10

# %%
# =============================================================================
# КОНФИГУРАЦИЯ
# =============================================================================

# Определяем корневую директорию
ROOT_DIR = Path("/home/evstigneva/nto25/baseline").resolve()
DATA_DIR = ROOT_DIR / "data"
RAW_DATA_DIR = Path("/home/evstigneva/Zagr")
INTERIM_DATA_DIR = DATA_DIR / "interim" 
PROCESSED_DATA_DIR = DATA_DIR / "processed"
OUTPUT_DIR = ROOT_DIR / "output"
MODEL_DIR = OUTPUT_DIR / "models"
SUBMISSION_DIR = OUTPUT_DIR / "submissions"
ALS_DIR = MODEL_DIR / "als"

# Создаем необходимые директории
for dir_path in [DATA_DIR, PROCESSED_DATA_DIR, MODEL_DIR, SUBMISSION_DIR, ALS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("✅ Директории созданы:")
print(f"   - Данные: {DATA_DIR}")
print(f"   - Модели: {MODEL_DIR}")
print(f"   - ALS модели: {ALS_DIR}")
print(f"   - Результаты: {SUBMISSION_DIR}")

# Проверяем наличие файлов данных
print("\n🔍 Проверка файлов данных:")
data_files = [
    TRAIN_FILENAME, TEST_FILENAME, USER_DATA_FILENAME, 
    BOOK_DATA_FILENAME, BOOK_GENRES_FILENAME, GENRES_FILENAME, BOOK_DESCRIPTIONS_FILENAME
]

for file_path in data_files:
    if Path(file_path).exists():
        print(f"   ✅ {Path(file_path).name} - найден")
    else:
        print(f"   ❌ {Path(file_path).name} - НЕ НАЙДЕН")

# --- ПАРАМЕТРЫ ---
RANDOM_STATE = 42
TARGET = COL_TARGET

# --- КОНФИГУРАЦИЯ ВРЕМЕННОГО РАЗДЕЛЕНИЯ ---
TEMPORAL_SPLIT_RATIO = 0.8

# --- КОНФИГУРАЦИЯ ОБУЧЕНИЯ ---
EARLY_STOPPING_ROUNDS = 50
MODEL_FILENAME = "model_with_toread.pkl"  # Универсальное имя

# --- ПАРАМЕТРЫ TF-IDF ---
TFIDF_MAX_FEATURES = 100  # Уменьшим для скорости
TFIDF_MIN_DF = 2
TFIDF_MAX_DF = 0.95
TFIDF_NGRAM_RANGE = (1, 1)

# --- ПАРАМЕТРЫ ALS ---
ALS_REGULARIZATION = 0.1
ALS_ITERATIONS = 10

# --- ФИЧИ ---
CAT_FEATURES = [
    COL_USER_ID,
    COL_BOOK_ID,
    COL_GENDER,
    COL_AUTHOR_ID,
    COL_LANGUAGE,
    COL_PUBLISHER,
    F_IS_IN_TOREAD_LIST,
]

# --- ПАРАМЕТРЫ МОДЕЛИ ---
LGB_PARAMS = {
    "objective": "rmse",
    "metric": "rmse",
    "n_estimators": 500,  # Уменьшим для скорости
    "learning_rate": 0.05,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "lambda_l1": 0.1,
    "lambda_l2": 0.1,
    "num_leaves": 31,
    "min_child_samples": 20,
    "verbose": -1,
    "n_jobs": -1,
    "seed": RANDOM_STATE,
    "boosting_type": "gbdt",
}

# Параметры для RandomForest (если LightGBM недоступен)
RF_PARAMS = {
    "n_estimators": 100,
    "max_depth": 20,
    "min_samples_split": 20,
    "random_state": RANDOM_STATE,
    "n_jobs": -1,
    "verbose": 0
}

# %%
# =============================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =============================================================================

def print_memory_usage(df: pd.DataFrame, name: str = "DataFrame"):
    """Выводит информацию об использовании памяти."""
    memory_mb = df.memory_usage(deep=True).sum() / 1024 ** 2
    print(f"{name}: {memory_mb:.2f} MB, {len(df):,} строк, {len(df.columns)} колонок")

def optimize_dataframe_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """Оптимизирует типы данных для уменьшения использования памяти."""
    df = df.copy()
    
    # Оптимизация числовых колонок
    for col in df.select_dtypes(include=['int64']).columns:
        if col in [COL_USER_ID, COL_BOOK_ID, COL_AUTHOR_ID]:
            df[col] = df[col].astype('int32')
        else:
            df[col] = pd.to_numeric(df[col], downcast='integer')
    
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    
    # Оптимизация категориальных колонок
    for col in CAT_FEATURES:
        if col in df.columns and df[col].nunique() < 0.5 * len(df):
            df[col] = df[col].astype('category')
    
    return df

# %%
# =============================================================================
# ФУНКЦИИ ОБРАБОТКИ ДАННЫХ
# =============================================================================

def load_all_interactions() -> pd.DataFrame:
    """Загружает ВСЕ взаимодействия из train.csv (и has_read=1, и has_read=0)."""
    print("Загрузка всех взаимодействий из train.csv...")
    
    dtype_spec = {
        COL_USER_ID: "int32",
        COL_BOOK_ID: "int32",
        COL_HAS_READ: "int8",
        COL_TARGET: "float32",
    }
    
    try:
        all_interactions = pd.read_csv(
            TRAIN_FILENAME,
            dtype=dtype_spec,
            parse_dates=[COL_TIMESTAMP],
        )
        
        print(f"Загружено {len(all_interactions):,} взаимодействий")
        print(f"Распределение has_read: {all_interactions[COL_HAS_READ].value_counts().to_dict()}")
        
        return all_interactions
    except Exception as e:
        print(f"⚠️  Ошибка загрузки данных: {e}")
        return pd.DataFrame()

def load_and_merge_data():
    """Загружает сырые данные и объединяет их в единый DataFrame."""
    print("Загрузка данных...")

    # Определяем типы данных для оптимизации памяти
    dtype_spec = {
        COL_USER_ID: "int32",
        COL_BOOK_ID: "int32",
        COL_TARGET: "float32",
        COL_GENDER: "category",
        COL_AGE: "float32",
        COL_AUTHOR_ID: "int32",
        COL_PUBLICATION_YEAR: "float32",
        COL_LANGUAGE: "category",
        COL_PUBLISHER: "category",
        COL_AVG_RATING: "float32",
        COL_GENRE_ID: "int16",
    }

    try:
        # Загружаем датасеты
        train_df = pd.read_csv(
            TRAIN_FILENAME,
            dtype={
                k: v
                for k, v in dtype_spec.items()
                if k in [COL_USER_ID, COL_BOOK_ID, COL_TARGET]
            },
            parse_dates=[COL_TIMESTAMP],
        )

        # Фильтруем тренировочные данные: только книги с рейтингом (has_read=1) ДЛЯ ОБУЧЕНИЯ МОДЕЛИ
        initial_count = len(train_df)
        train_df_filtered = train_df[train_df[COL_HAS_READ] == 1].copy()
        filtered_count = len(train_df_filtered)
        print(f"Отфильтровано тренировочных данных: {initial_count} -> {filtered_count} строк (только has_read=1)")
        
        test_df = pd.read_csv(
            TEST_FILENAME,
            dtype={k: v for k, v in dtype_spec.items() if k in [COL_USER_ID, COL_BOOK_ID]},
        )
        user_data_df = pd.read_csv(
            USER_DATA_FILENAME,
            dtype={
                k: v for k, v in dtype_spec.items() if k in [COL_USER_ID, COL_GENDER, COL_AGE]
            },
        )
        book_data_df = pd.read_csv(
            BOOK_DATA_FILENAME,
            dtype={
                k: v
                for k, v in dtype_spec.items()
                if k
                in [
                    COL_BOOK_ID,
                    COL_AUTHOR_ID,
                    COL_PUBLICATION_YEAR,
                    COL_LANGUAGE,
                    COL_AVG_RATING,
                    COL_PUBLISHER,
                ]
            },
        )
        book_genres_df = pd.read_csv(
            BOOK_GENRES_FILENAME,
            dtype={k: v for k, v in dtype_spec.items() if k in [COL_BOOK_ID, COL_GENRE_ID]},
        )
        genres_df = pd.read_csv(GENRES_FILENAME)
        book_descriptions_df = pd.read_csv(
            BOOK_DESCRIPTIONS_FILENAME,
            dtype={COL_BOOK_ID: "int32"},
        )

        print("Данные загружены. Объединение датасетов...")

        # Объединяем train (только has_read=1) и test
        train_df_filtered[COL_SOURCE] = VAL_SOURCE_TRAIN
        test_df[COL_SOURCE] = VAL_SOURCE_TEST
        combined_df = pd.concat([train_df_filtered, test_df], ignore_index=True, sort=False)

        # Добавляем метаданные пользователей
        combined_df = combined_df.merge(user_data_df, on=COL_USER_ID, how="left")

        # Удаляем дубликаты из book_data_df перед объединением
        book_data_df = book_data_df.drop_duplicates(subset=[COL_BOOK_ID])
        combined_df = combined_df.merge(book_data_df, on=COL_BOOK_ID, how="left")

        print(f"Размер объединенных данных: {combined_df.shape}")
        
        # Возвращаем также все взаимодействия для создания признаков
        return combined_df, book_genres_df, book_descriptions_df, train_df_filtered
    
    except Exception as e:
        print(f"⚠️  Ошибка загрузки данных: {e}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

# %%
# =============================================================================
# ФУНКЦИИ ДЛЯ СТРАТЕГИИ 2 (has_read=0) - УПРОЩЕННАЯ ВЕРСИЯ
# =============================================================================

def create_toread_basic_features(df: pd.DataFrame, all_interactions: pd.DataFrame) -> pd.DataFrame:
    """Создает базовые фичи из списка 'на прочтение' (упрощенная версия)."""
    print("Создание базовых фич из списка 'на прочтение'...")
    
    if all_interactions.empty:
        print("  ⚠️  Нет данных о взаимодействиях. Пропускаем создание фич.")
        return df
    
    df = df.copy()
    
    # 1. Флаг: находится ли книга в списке "на прочтение" у данного пользователя
    print("  - Флаг is_in_toread_list...")
    
    try:
        # Создаем множество пар (user_id, book_id) из списка "на прочтение"
        toread_interactions = all_interactions[all_interactions[COL_HAS_READ] == 0]
        
        if not toread_interactions.empty:
            toread_pairs = set(zip(toread_interactions[COL_USER_ID], toread_interactions[COL_BOOK_ID]))
            
            # Проверяем наличие каждой пары в множестве
            df[F_IS_IN_TOREAD_LIST] = df.apply(
                lambda row: 1 if (row[COL_USER_ID], row[COL_BOOK_ID]) in toread_pairs else 0, 
                axis=1
            )
        else:
            df[F_IS_IN_TOREAD_LIST] = 0
    
    except Exception as e:
        print(f"  ⚠️  Ошибка создания флага is_in_toread_list: {e}")
        df[F_IS_IN_TOREAD_LIST] = 0
    
    # 2. Простые статистики по пользователям
    print("  - Простые статистики...")
    
    try:
        # Количество книг в списке "на прочтение" для каждого пользователя
        if not toread_interactions.empty:
            user_toread_count = toread_interactions.groupby(COL_USER_ID)[COL_BOOK_ID].count().reset_index()
            user_toread_count.columns = [COL_USER_ID, F_USER_TOREAD_COUNT]
            df = df.merge(user_toread_count, on=COL_USER_ID, how="left")
        else:
            df[F_USER_TOREAD_COUNT] = 0
        
        # Количество прочитанных книг для каждого пользователя
        read_interactions = all_interactions[all_interactions[COL_HAS_READ] == 1]
        if not read_interactions.empty:
            user_read_count = read_interactions.groupby(COL_USER_ID)[COL_BOOK_ID].count().reset_index()
            user_read_count.columns = [COL_USER_ID, 'user_read_count_temp']
            df = df.merge(user_read_count, on=COL_USER_ID, how="left")
        else:
            df['user_read_count_temp'] = 0
        
        # Соотношение прочитанных к книгам в списке
        df[F_USER_READ_TOREAD_RATIO] = df.apply(
            lambda x: x['user_read_count_temp'] / max(x.get(F_USER_TOREAD_COUNT, 1), 1),
            axis=1
        )
        
        # Удаляем временную колонку
        if 'user_read_count_temp' in df.columns:
            df = df.drop(columns=['user_read_count_temp'])
    
    except Exception as e:
        print(f"  ⚠️  Ошибка создания статистик пользователей: {e}")
    
    # 3. Простые статистики по книгам
    try:
        if not toread_interactions.empty:
            book_toread_count = toread_interactions.groupby(COL_BOOK_ID)[COL_USER_ID].count().reset_index()
            book_toread_count.columns = [COL_BOOK_ID, F_BOOK_TOREAD_COUNT]
            df = df.merge(book_toread_count, on=COL_BOOK_ID, how="left")
        else:
            df[F_BOOK_TOREAD_COUNT] = 0
        
        if not read_interactions.empty:
            book_read_count = read_interactions.groupby(COL_BOOK_ID)[COL_USER_ID].count().reset_index()
            book_read_count.columns = [COL_BOOK_ID, 'book_read_count_temp']
            df = df.merge(book_read_count, on=COL_BOOK_ID, how="left")
        else:
            df['book_read_count_temp'] = 0
        
        # Соотношение прочтений к добавлениям в список
        df[F_BOOK_READ_TOREAD_RATIO] = df.apply(
            lambda x: x['book_read_count_temp'] / max(x.get(F_BOOK_TOREAD_COUNT, 1), 1),
            axis=1
        )
        
        # Удаляем временную колонку
        if 'book_read_count_temp' in df.columns:
            df = df.drop(columns=['book_read_count_temp'])
    
    except Exception as e:
        print(f"  ⚠️  Ошибка создания статистик книг: {e}")
    
    # Заполняем пропуски
    fill_values = {
        F_USER_TOREAD_COUNT: 0,
        F_USER_READ_TOREAD_RATIO: 0,
        F_BOOK_TOREAD_COUNT: 0,
        F_BOOK_READ_TOREAD_RATIO: 0,
        F_IS_IN_TOREAD_LIST: 0,
    }
    
    for col, fill_value in fill_values.items():
        if col in df.columns:
            df[col] = df[col].fillna(fill_value)
    
    # Подсчитываем добавленные фичи
    added_features = sum(1 for col in df.columns if any(keyword in col.lower() for keyword in ['toread', 'is_in']))
    print(f"  Добавлено {added_features} фич из списка 'на прочтение'")
    
    return df

def create_simple_als_features(df: pd.DataFrame, all_interactions: pd.DataFrame) -> pd.DataFrame:
    """Упрощенная версия ALS фичей."""
    if not ALS_AVAILABLE or all_interactions.empty:
        print("  ⚠️  ALS фичи отключены или нет данных")
        return df
    
    print("Создание упрощенных ALS фичей...")
    
    try:
        # Простой подсчет взаимодействий
        user_item_counts = all_interactions.groupby([COL_USER_ID, COL_BOOK_ID]).size().reset_index(name='interaction_count')
        
        # Для каждого пользователя: среднее количество взаимодействий с книгами
        user_avg_interactions = user_item_counts.groupby(COL_USER_ID)['interaction_count'].mean().reset_index()
        user_avg_interactions.columns = [COL_USER_ID, F_ALS_USER_SCORE]
        
        # Для каждой книги: среднее количество взаимодействий с пользователями
        book_avg_interactions = user_item_counts.groupby(COL_BOOK_ID)['interaction_count'].mean().reset_index()
        book_avg_interactions.columns = [COL_BOOK_ID, F_ALS_BOOK_SCORE]
        
        # Объединяем с основным датафреймом
        df = df.merge(user_avg_interactions, on=COL_USER_ID, how="left")
        df = df.merge(book_avg_interactions, on=COL_BOOK_ID, how="left")
        
        # Скалярное произведение (упрощенное)
        df[F_ALS_DOT_PRODUCT] = df[F_ALS_USER_SCORE] * df[F_ALS_BOOK_SCORE]
        
        # Заполняем пропуски
        for col in [F_ALS_USER_SCORE, F_ALS_BOOK_SCORE, F_ALS_DOT_PRODUCT]:
            if col in df.columns:
                df[col] = df[col].fillna(0)
        
        print("  Упрощенные ALS фичи добавлены")
        
    except Exception as e:
        print(f"  ⚠️  Ошибка создания ALS фичей: {e}")
    
    return df

# %%
# =============================================================================
# ОСНОВНЫЕ ФУНКЦИИ ФИЧЕЙ
# =============================================================================

def add_aggregate_features(df, train_df, all_interactions):
    """Вычисляет и добавляет агрегированные фичи пользователей, книг и авторов."""
    print("Добавление агрегированных фич...")

    if train_df.empty or all_interactions.empty:
        print("  ⚠️  Нет тренировочных данных. Пропускаем агрегаты.")
        return df

    try:
        # Используем только has_read=1 для агрегатов
        read_interactions = all_interactions[all_interactions[COL_HAS_READ] == 1]
        
        if read_interactions.empty:
            print("  ⚠️  Нет данных с has_read=1. Пропускаем агрегаты.")
            return df
        
        # Агрегаты по пользователям (только has_read=1)
        user_agg = read_interactions.groupby(COL_USER_ID)[TARGET].agg(["mean", "count"]).reset_index()
        user_agg.columns = [
            COL_USER_ID,
            F_USER_MEAN_RATING,
            F_USER_RATINGS_COUNT,
        ]

        # Агрегаты по книгам (только has_read=1)
        book_agg = read_interactions.groupby(COL_BOOK_ID)[TARGET].agg(["mean", "count"]).reset_index()
        book_agg.columns = [
            COL_BOOK_ID,
            F_BOOK_MEAN_RATING,
            F_BOOK_RATINGS_COUNT,
        ]

        # Объединяем агрегаты с основным датафреймом
        df = df.merge(user_agg, on=COL_USER_ID, how="left")
        df = df.merge(book_agg, on=COL_BOOK_ID, how="left")
        
    except Exception as e:
        print(f"  ⚠️  Ошибка создания агрегатных фич: {e}")
    
    return df

def add_genre_features(df, book_genres_df):
    """Вычисляет и добавляет количество жанров для каждой книги."""
    print("Добавление фич жанров...")
    
    if book_genres_df.empty:
        print("  ⚠️  Нет данных о жанрах. Пропускаем.")
        return df
    
    try:
        genre_counts = book_genres_df.groupby(COL_BOOK_ID)[COL_GENRE_ID].count().reset_index()
        genre_counts.columns = [
            COL_BOOK_ID,
            F_BOOK_GENRES_COUNT,
        ]
        df = df.merge(genre_counts, on=COL_BOOK_ID, how="left")
        df[F_BOOK_GENRES_COUNT] = df[F_BOOK_GENRES_COUNT].fillna(0)
    except Exception as e:
        print(f"  ⚠️  Ошибка создания жанровых фич: {e}")
    
    return df

def add_text_features(df, train_df, descriptions_df):
    """Добавляет TF-IDF фичи из описаний книг."""
    print("Добавление текстовых фич (TF-IDF)...")

    if descriptions_df.empty:
        print("  ⚠️  Нет описаний книг. Пропускаем TF-IDF.")
        return df

    vectorizer_path = MODEL_DIR / TFIDF_VECTORIZER_FILENAME

    try:
        # Получаем уникальные книги из тренировочного набора
        train_books = train_df[COL_BOOK_ID].unique()

        # Извлекаем описания только для тренировочных книг
        train_descriptions = descriptions_df[descriptions_df[COL_BOOK_ID].isin(train_books)].copy()
        train_descriptions[COL_DESCRIPTION] = train_descriptions[COL_DESCRIPTION].fillna("")

        # Проверяем существование векторайзера
        if vectorizer_path.exists():
            print(f"Загрузка существующего векторайзера из {vectorizer_path}")
            vectorizer = joblib.load(vectorizer_path)
        else:
            # Обучаем векторайзер только на тренировочных описаниях
            print("Обучение TF-IDF векторайзера на тренировочных описаниях...")
            vectorizer = TfidfVectorizer(
                max_features=TFIDF_MAX_FEATURES,
                min_df=TFIDF_MIN_DF,
                max_df=TFIDF_MAX_DF,
                ngram_range=TFIDF_NGRAM_RANGE,
            )
            vectorizer.fit(train_descriptions[COL_DESCRIPTION])
            # Сохраняем векторайзер для использования в предсказании
            joblib.dump(vectorizer, vectorizer_path)
            print(f"Векторайзер сохранен в {vectorizer_path}")

        # Трансформируем все описания книг
        all_descriptions = descriptions_df[[COL_BOOK_ID, COL_DESCRIPTION]].copy()
        all_descriptions[COL_DESCRIPTION] = all_descriptions[COL_DESCRIPTION].fillna("")

        # Создаем маппинг book_id -> description
        description_map = dict(
            zip(all_descriptions[COL_BOOK_ID], all_descriptions[COL_DESCRIPTION])
        )

        # Получаем описания для книг в df
        df_descriptions = df[COL_BOOK_ID].map(description_map).fillna("")

        # Трансформируем в TF-IDF фичи
        tfidf_matrix = vectorizer.transform(df_descriptions)

        # Конвертируем разреженную матрицу в DataFrame
        tfidf_feature_names = [f"tfidf_{i}" for i in range(tfidf_matrix.shape[1])]
        tfidf_df = pd.DataFrame(
            tfidf_matrix.toarray(),
            columns=tfidf_feature_names,
            index=df.index,
        )

        # Объединяем TF-IDF фичи с основным DataFrame
        df_with_tfidf = pd.concat([df.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

        print(f"Добавлено {len(tfidf_feature_names)} TF-IDF фич.")
        return df_with_tfidf
    
    except Exception as e:
        print(f"  ⚠️  Ошибка создания TF-IDF фич: {e}")
        return df

def handle_missing_values(df, train_df):
    """Заполняет пропущенные значения используя определенную стратегию."""
    print("Обработка пропущенных значений...")

    try:
        # Вычисляем глобальное среднее из тренировочных данных для заполнения
        if not train_df.empty and TARGET in train_df.columns:
            global_mean = train_df[TARGET].mean()
        else:
            global_mean = 5.0

        # Заполняем возраст медианой
        if COL_AGE in df.columns:
            age_median = df[COL_AGE].median()
            df[COL_AGE] = df[COL_AGE].fillna(age_median)

        # Заполняем агрегированные фичи для "холодного старта"
        for col in [F_USER_MEAN_RATING, F_BOOK_MEAN_RATING, COL_AVG_RATING]:
            if col in df.columns:
                df[col] = df[col].fillna(global_mean)

        # Заполняем счетчики нулями
        count_cols = [
            F_USER_RATINGS_COUNT, F_BOOK_RATINGS_COUNT,
            F_USER_TOREAD_COUNT, F_BOOK_TOREAD_COUNT,
            F_BOOK_GENRES_COUNT
        ]
        for col in count_cols:
            if col in df.columns:
                df[col] = df[col].fillna(0)

        # Заполняем соотношения нулями
        ratio_cols = [
            F_USER_READ_TOREAD_RATIO, F_BOOK_READ_TOREAD_RATIO,
        ]
        for col in ratio_cols:
            if col in df.columns:
                df[col] = df[col].fillna(0)

        # Заполняем бинарные фичи
        if F_IS_IN_TOREAD_LIST in df.columns:
            df[F_IS_IN_TOREAD_LIST] = df[F_IS_IN_TOREAD_LIST].fillna(0).astype(int)

        # Заполняем ALS фичи нулями
        als_cols = [F_ALS_USER_SCORE, F_ALS_BOOK_SCORE, F_ALS_DOT_PRODUCT]
        for col in als_cols:
            if col in df.columns:
                df[col] = df[col].fillna(0.0)

        # Заполняем TF-IDF фичи нулями
        tfidf_cols = [col for col in df.columns if col.startswith("tfidf_")]
        for col in tfidf_cols:
            if col in df.columns:
                df[col] = df[col].fillna(0.0)

        # Заполняем оставшиеся категориальные фичи специальным значением
        for col in df.columns:
            if col in CAT_FEATURES:
                if df[col].dtype.name in ("category", "object") and df[col].isna().any():
                    df[col] = df[col].astype(str).fillna(MISSING_CAT_VALUE).astype("category")
                elif pd.api.types.is_numeric_dtype(df[col].dtype) and df[col].isna().any():
                    df[col] = df[col].fillna(MISSING_NUM_VALUE)
    
    except Exception as e:
        print(f"  ⚠️  Ошибка обработки пропущенных значений: {e}")
    
    return df

def create_features(df, all_interactions, book_genres_df, descriptions_df, train_df):
    """Запускает полный пайплайн инженерии фич (упрощенная версия)."""
    print("Запуск пайплайна инженерии фич...")
    
    if df.empty:
        print("  ⚠️  Пустой датафрейм. Пропускаем инженерию фич.")
        return df
    
    # 1. Базовые фичи
    df = add_genre_features(df, book_genres_df)
    
    # 2. Фичи из списка "на прочтение" (СТРАТЕГИЯ 2)
    df = create_toread_basic_features(df, all_interactions)
    
    # 3. ALS фичи (упрощенные)
    df = create_simple_als_features(df, all_interactions)
    
    # 4. Текстовые фичи
    df = add_text_features(df, train_df, descriptions_df)
    
    # 5. Обработка пропущенных значений
    df = handle_missing_values(df, train_df)

    print("Инженерия фич завершена.")
    return df

# %%
# =============================================================================
# ФУНКЦИИ ВРЕМЕННОГО РАЗДЕЛЕНИЯ
# =============================================================================

def temporal_split_by_date(df, split_date, timestamp_col=COL_TIMESTAMP):
    """Разделяет DataFrame на тренировочную и валидационную выборки по абсолютной дате."""
    if timestamp_col not in df.columns:
        print(f"⚠️  Колонка с временными метками '{timestamp_col}' не найдена. Используем случайное разделение.")
        np.random.seed(RANDOM_STATE)
        mask = np.random.rand(len(df)) < TEMPORAL_SPLIT_RATIO
        return pd.Series(mask), pd.Series(~mask)

    try:
        # Убеждаемся, что временная метка в datetime формате
        if not pd.api.types.is_datetime64_any_dtype(df[timestamp_col]):
            df = df.copy()
            df[timestamp_col] = pd.to_datetime(df[timestamp_col])

        # Разделяем по порогу даты
        train_mask = df[timestamp_col] <= split_date
        val_mask = df[timestamp_col] > split_date

        # Валидационные проверки
        if train_mask.sum() == 0:
            print(f"⚠️  Не найдено записей с временной меткой <= {split_date}.")
            return pd.Series([True] * len(df)), pd.Series([False] * len(df))

        if val_mask.sum() == 0:
            print(f"⚠️  Не найдено записей с временной метка > {split_date}.")
            return pd.Series([True] * len(df)), pd.Series([False] * len(df))

        return train_mask, val_mask
    
    except Exception as e:
        print(f"⚠️  Ошибка временного разделения: {e}")
        # Возвращаем случайное разделение в случае ошибки
        np.random.seed(RANDOM_STATE)
        mask = np.random.rand(len(df)) < TEMPORAL_SPLIT_RATIO
        return pd.Series(mask), pd.Series(~mask)

def get_split_date_from_ratio(df, ratio, timestamp_col=COL_TIMESTAMP):
    """Вычисляет дату разделения на основе соотношения точек данных."""
    if not 0 < ratio < 1:
        ratio = 0.8  # Значение по умолчанию

    if timestamp_col not in df.columns:
        print(f"⚠️  Колонка с временными метками '{timestamp_col}' не найдена.")
        return None

    try:
        # Убеждаемся, что временная метка в datetime формате
        if not pd.api.types.is_datetime64_any_dtype(df[timestamp_col]):
            df = df.copy()
            df[timestamp_col] = pd.to_datetime(df[timestamp_col])

        # Вычисляем порог на основе соотношения
        sorted_timestamps = df[timestamp_col].sort_values()
        threshold_index = int(len(sorted_timestamps) * ratio)

        return sorted_timestamps.iloc[threshold_index]
    
    except Exception as e:
        print(f"⚠️  Ошибка вычисления даты разделения: {e}")
        return None

# %%
# =============================================================================
# ОСНОВНЫЕ ФУНКЦИИ ПАЙПЛАЙНА
# =============================================================================

def prepare_data():
    """Обрабатывает сырые данные и сохраняет подготовленные фичи в processed директорию."""
    print("=" * 60)
    print("ПАЙПЛАЙН ПОДГОТОВКИ ДАННЫХ С ФИЧАМИ ИЗ has_read=0")
    print("=" * 60)

    # Проверяем наличие необходимых файлов
    required_files = [TRAIN_FILENAME, TEST_FILENAME]
    
    missing_files = [f for f in required_files if not Path(f).exists()]
    if missing_files:
        print(f"⚠️  Отсутствуют необходимые файлы: {missing_files}")
        return pd.DataFrame()

    # Загружаем и объединяем сырые данные
    merged_df, book_genres_df, descriptions_df, train_df_filtered = load_and_merge_data()
    
    if merged_df.empty:
        print("⚠️  Не удалось загрузить данные. Пропускаем подготовку.")
        return pd.DataFrame()

    # Загружаем все взаимодействия для создания фичей
    print("\nЗагрузка всех взаимодействий для создания фичей...")
    all_interactions = load_all_interactions()

    # Применяем инженерию фич
    print("\nПрименение инженерии фич...")
    featured_df = create_features(
        merged_df, 
        all_interactions, 
        book_genres_df, 
        descriptions_df,
        train_df_filtered
    )

    # Оптимизируем использование памяти
    featured_df = optimize_dataframe_dtypes(featured_df)

    # Определяем путь вывода
    processed_path = PROCESSED_DATA_DIR / PROCESSED_DATA_FILENAME

    # Сохраняем обработанные данные как parquet для эффективности
    print(f"\nСохранение обработанных данных в {processed_path}...")
    try:
        featured_df.to_parquet(processed_path, index=False, engine="pyarrow", compression="snappy")
        print("Обработанные данные успешно сохранены!")
    except Exception as e:
        print(f"⚠️  Ошибка сохранения данных: {e}")
        # Пробуем сохранить как CSV
        csv_path = processed_path.with_suffix('.csv')
        featured_df.to_csv(csv_path, index=False)
        print(f"Данные сохранены как CSV: {csv_path}")

    # Печатаем статистику
    if not featured_df.empty:
        train_rows = len(featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TRAIN])
        test_rows = len(featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TEST])
        total_features = len(featured_df.columns)
        
        print("\n" + "=" * 60)
        print("ПОДГОТОВКА ДАННЫХ ЗАВЕРШЕНА!")
        print("=" * 60)
        print(f"  - Тренировочные строки: {train_rows:,}")
        print(f"  - Тестовые строки: {test_rows:,}")
        print(f"  - Всего фич: {total_features}")
        print(f"  - Выходной файл: {processed_path}")
    else:
        print("⚠️  Обработанные данные пусты.")
    
    return featured_df

def train():
    """Запускает пайплайн обучения модели с временным разделением."""
    print("=" * 60)
    print("ОБУЧЕНИЕ МОДЕЛИ С ФИЧАМИ ИЗ has_read=0")
    print("=" * 60)
    
    # Загружаем подготовленные данные
    processed_path = PROCESSED_DATA_DIR / PROCESSED_DATA_FILENAME

    if not processed_path.exists():
        # Пробуем найти CSV версию
        csv_path = processed_path.with_suffix('.csv')
        if csv_path.exists():
            print(f"Загрузка подготовленных данных из {csv_path}...")
            featured_df = pd.read_csv(csv_path)
        else:
            print(f"⚠️  Обработанные данные не найдены. Запускаем prepare_data()...")
            featured_df = prepare_data()
            if featured_df.empty:
                print("⚠️  Не удалось подготовить данные. Пропускаем обучение.")
                return None, 0, 0, pd.DataFrame()
    else:
        print(f"Загрузка подготовленных данных из {processed_path}...")
        featured_df = pd.read_parquet(processed_path, engine="pyarrow")
    
    if featured_df.empty:
        print("⚠️  Загружен пустой датафрейм. Пропускаем обучение.")
        return None, 0, 0, pd.DataFrame()
    
    print_memory_usage(featured_df, "Загруженные данные")
    print(f"Загружено {len(featured_df):,} строк с {len(featured_df.columns)} фичами")

    # Разделяем тренировочные и тестовые наборы
    train_set = featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TRAIN].copy()
    
    if train_set.empty:
        print("⚠️  Нет тренировочных данных. Пропускаем обучение.")
        return None, 0, 0, pd.DataFrame()

    # Проверяем наличие колонки временных меток
    if COL_TIMESTAMP not in train_set.columns:
        print(f"⚠️  Колонка с временными метками '{COL_TIMESTAMP}' не найдена. Используем случайное разделение.")
        np.random.seed(RANDOM_STATE)
        mask = np.random.rand(len(train_set)) < TEMPORAL_SPLIT_RATIO
        train_split = train_set[mask].copy()
        val_split = train_set[~mask].copy()
    else:
        try:
            # Выполняем временное разделение
            print(f"\nВыполнение временного разделения с соотношением {TEMPORAL_SPLIT_RATIO}...")
            split_date = get_split_date_from_ratio(train_set, TEMPORAL_SPLIT_RATIO, COL_TIMESTAMP)
            
            if split_date is None:
                print("⚠️  Не удалось вычислить дату разделения. Используем случайное разделение.")
                np.random.seed(RANDOM_STATE)
                mask = np.random.rand(len(train_set)) < TEMPORAL_SPLIT_RATIO
                train_split = train_set[mask].copy()
                val_split = train_set[~mask].copy()
            else:
                print(f"Дата разделения: {split_date}")

                train_mask, val_mask = temporal_split_by_date(train_set, split_date, COL_TIMESTAMP)

                # Разделяем данные
                train_split = train_set[train_mask].copy()
                val_split = train_set[val_mask].copy()
            
        except Exception as e:
            print(f"⚠️  Ошибка временного разделения: {e}")
            # Случайное разделение
            np.random.seed(RANDOM_STATE)
            mask = np.random.rand(len(train_set)) < TEMPORAL_SPLIT_RATIO
            train_split = train_set[mask].copy()
            val_split = train_set[~mask].copy()

    print(f"Тренировочное разделение: {len(train_split):,} строк")
    print(f"Валидационное разделение: {len(val_split):,} строк")

    # Загружаем все взаимодействия для создания агрегатов
    print("\nЗагрузка всех взаимодействий для создания агрегатов...")
    all_interactions = load_all_interactions()
    
    # Вычисляем агрегированные фичи
    print("\nВычисление агрегированных фич...")
    train_split_with_agg = add_aggregate_features(train_split.copy(), train_split, all_interactions)
    val_split_with_agg = add_aggregate_features(val_split.copy(), train_split, all_interactions)

    # Обрабатываем пропущенные значения
    print("Обработка пропущенных значений...")
    train_split_final = handle_missing_values(train_split_with_agg, train_split)
    val_split_final = handle_missing_values(val_split_with_agg, train_split)

    # Определяем фичи (X) и таргет (y)
    exclude_cols = [
        COL_SOURCE,
        TARGET,
        COL_PREDICTION,
        COL_TIMESTAMP,
        COL_HAS_READ,
    ]
    
    # Исключаем колонки, которых нет в данных
    available_cols = [col for col in train_split_final.columns if col not in exclude_cols]
    
    # Исключаем object колонки
    non_feature_object_cols = train_split_final[available_cols].select_dtypes(include=["object"]).columns.tolist()
    features = [f for f in available_cols if f not in non_feature_object_cols]

    # Проверяем, что у нас есть фичи
    if not features:
        print("⚠️  Нет фич для обучения. Пропускаем.")
        return None, 0, 0, pd.DataFrame()

    X_train = train_split_final[features]
    y_train = train_split_final[TARGET]
    X_val = val_split_final[features]
    y_val = val_split_final[TARGET]

    print(f"\nТренировочные фичи: {len(features)}")
    print(f"Размер X_train: {X_train.shape}")
    print(f"Размер X_val: {X_val.shape}")

    # Обучаем модель
    print("\n" + "=" * 60)
    print("ОБУЧЕНИЕ МОДЕЛИ")
    print("=" * 60)
    
    try:
        if LGB_AVAILABLE:
            print("Используем LightGBM...")
            model = lgb.LGBMRegressor(**LGB_PARAMS)

            # Обновляем параметры fit с колбэком ранней остановки
            fit_params = {"eval_metric": "rmse"}
            
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_val, y_val)],
                eval_metric=fit_params["eval_metric"],
                callbacks=[lgb.early_stopping(stopping_rounds=EARLY_STOPPING_ROUNDS)],
                verbose=100,
            )
        else:
            print("Используем RandomForestRegressor...")
            from sklearn.ensemble import RandomForestRegressor
            model = RandomForestRegressor(**RF_PARAMS)
            model.fit(X_train, y_train)

        # Оцениваем модель
        val_preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        mae = mean_absolute_error(y_val, val_preds)
        
        print("\n" + "=" * 60)
        print("РЕЗУЛЬТАТЫ ОБУЧЕНИЯ")
        print("=" * 60)
        print(f"✅ Валидационный RMSE: {rmse:.4f}")
        print(f"✅ Валидационный MAE: {mae:.4f}")

        # Анализ важности фич
        print("\n" + "=" * 60)
        print("ТОП-20 ВАЖНЕЙШИХ ФИЧ")
        print("=" * 60)
        
        if hasattr(model, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature': features,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False).head(20)
            
            for i, row in feature_importance.iterrows():
                print(f"  {i+1:2d}. {row['feature'][:50]:50s} - {row['importance']:8.1f}")
        else:
            print("  ⚠️  Модель не поддерживает feature_importances_")
            feature_importance = pd.DataFrame()

        # Сохраняем обученную модель
        model_path = MODEL_DIR / MODEL_FILENAME
        joblib.dump(model, model_path)
        print(f"\n✅ Модель сохранена в {model_path}")

        # Сохраняем список фич
        features_path = MODEL_DIR / "model_features.pkl"
        joblib.dump(features, features_path)
        print(f"✅ Список фич сохранен в {features_path}")

        print("\n" + "=" * 60)
        print("ОБУЧЕНИЕ ЗАВЕРШЕНО")
        print("=" * 60)
        
        return model, rmse, mae, feature_importance
    
    except Exception as e:
        print(f"⚠️  Ошибка обучения модели: {e}")
        # Пробуем простую линейную модель
        try:
            print("Пробуем LinearRegression...")
            from sklearn.linear_model import LinearRegression
            model = LinearRegression()
            model.fit(X_train, y_train)
            
            val_preds = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, val_preds))
            mae = mean_absolute_error(y_val, val_preds)
            
            print(f"✅ LinearRegression RMSE: {rmse:.4f}")
            print(f"✅ LinearRegression MAE: {mae:.4f}")
            
            # Сохраняем модель
            model_path = MODEL_DIR / MODEL_FILENAME
            joblib.dump(model, model_path)
            print(f"✅ Модель сохранена в {model_path}")
            
            # Сохраняем список фич
            features_path = MODEL_DIR / "model_features.pkl"
            joblib.dump(features, features_path)
            
            return model, rmse, mae, pd.DataFrame()
        except Exception as e2:
            print(f"⚠️  Ошибка обучения LinearRegression: {e2}")
            return None, 0, 0, pd.DataFrame()

def predict():
    """Генерирует и сохраняет предсказания для тестового набора."""
    print("=" * 60)
    print("ПРЕДСКАЗАНИЕ С ФИЧАМИ ИЗ has_read=0")
    print("=" * 60)
    
    # Загружаем подготовленные данные
    processed_path = PROCESSED_DATA_DIR / PROCESSED_DATA_FILENAME

    if not processed_path.exists():
        # Пробуем найти CSV версию
        csv_path = processed_path.with_suffix('.csv')
        if csv_path.exists():
            print(f"Загрузка подготовленных данных из {csv_path}...")
            featured_df = pd.read_csv(csv_path)
        else:
            print(f"⚠️  Обработанные данные не найдены. Запускаем prepare_data()...")
            featured_df = prepare_data()
            if featured_df.empty:
                print("⚠️  Не удалось подготовить данные. Пропускаем предсказание.")
                return pd.DataFrame(), np.array([])
    else:
        print(f"Загрузка подготовленных данных из {processed_path}...")
        featured_df = pd.read_parquet(processed_path, engine="pyarrow")
    
    if featured_df.empty:
        print("⚠️  Загружен пустой датафрейм. Пропускаем предсказание.")
        return pd.DataFrame(), np.array([])
    
    print_memory_usage(featured_df, "Загруженные данные")
    print(f"Загружено {len(featured_df):,} строк с {len(featured_df.columns)} фичами")

    # Разделяем тренировочные и тестовые наборы
    train_set = featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TRAIN].copy()
    test_set = featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TEST].copy()
    
    if test_set.empty:
        print("⚠️  Нет тестовых данных. Пропускаем предсказание.")
        return pd.DataFrame(), np.array([])

    print(f"Тренировочный набор: {len(train_set):,} строк")
    print(f"Тестовый набор: {len(test_set):,} строк")

    # Загружаем все взаимодействия для создания агрегатов
    print("\nЗагрузка всех взаимодействий для создания агрегатов...")
    all_interactions = load_all_interactions()
    
    # Для тестовых предсказаний используем ВСЕ тренировочные данные
    print("Вычисление агрегированных фич на всех тренировочных данных...")
    test_set_with_agg = add_aggregate_features(test_set.copy(), train_set, all_interactions)

    # Обрабатываем пропущенные значения
    print("Обработка пропущенных значений...")
    test_set_final = handle_missing_values(test_set_with_agg, train_set)

    # Загружаем список фич из тренировки
    features_path = MODEL_DIR / "model_features.pkl"
    if not features_path.exists():
        print(f"⚠️  Список фич не найден. Создаем новый...")
        # Используем все фичи кроме служебных
        exclude_cols = [COL_SOURCE, TARGET, COL_PREDICTION, COL_TIMESTAMP, COL_HAS_READ]
        features = [col for col in test_set_final.columns if col not in exclude_cols]
        # Исключаем object колонки
        non_feature_object_cols = test_set_final[features].select_dtypes(include=["object"]).columns.tolist()
        features = [f for f in features if f not in non_feature_object_cols]
        # Сохраняем для будущего использования
        joblib.dump(features, features_path)
    else:
        features = joblib.load(features_path)
        print(f"Загружено {len(features)} фич из обученной модели")

    # Проверяем, что все фичи есть в тестовых данных
    missing_features = [f for f in features if f not in test_set_final.columns]
    if missing_features:
        print(f"⚠️  Отсутствуют фичи: {missing_features[:5]}...")
        print("Добавляем недостающие фичи с нулевыми значениями...")
        for f in missing_features:
            test_set_final[f] = 0

    X_test = test_set_final[features]
    print(f"Фичи для предсказания: {X_test.shape}")

    # Загружаем обученную модель
    model_path = MODEL_DIR / MODEL_FILENAME
    if not model_path.exists():
        print(f"⚠️  Модель не найдена в {model_path}. Запускаем обучение...")
        model, _, _, _ = train()
        if model is None:
            print("⚠️  Не удалось обучить модель. Создаем базовые предсказания...")
            # Создаем простые предсказания (глобальное среднее)
            train_target_mean = train_set[TARGET].mean() if not train_set.empty and TARGET in train_set.columns else 5.0
            test_preds = np.full(len(test_set), train_target_mean)
            clipped_preds = np.clip(test_preds, PREDICTION_MIN_VALUE, PREDICTION_MAX_VALUE)
            
            # Создаем файл submission
            submission_df = test_set[[COL_USER_ID, COL_BOOK_ID]].copy()
            submission_df[COL_PREDICTION] = clipped_preds
            
            submission_path = SUBMISSION_DIR / SUBMISSION_FILENAME
            submission_df.to_csv(submission_path, index=False)
            
            print(f"✅ Базовые предсказания созданы: {submission_path}")
            return submission_df, clipped_preds
    else:
        print(f"\nЗагрузка модели из {model_path}...")
        model = joblib.load(model_path)

    # Генерируем предсказания
    print("Генерация предсказаний...")
    test_preds = model.predict(X_test)

    # Обрезаем предсказания до валидного диапазона рейтинга [0, 10]
    clipped_preds = np.clip(test_preds, PREDICTION_MIN_VALUE, PREDICTION_MAX_VALUE)

    # Создаем файл submission
    submission_df = test_set[[COL_USER_ID, COL_BOOK_ID]].copy()
    submission_df[COL_PREDICTION] = clipped_preds

    submission_path = SUBMISSION_DIR / SUBMISSION_FILENAME
    submission_df.to_csv(submission_path, index=False)
    
    print("\n" + "=" * 60)
    print("ПРЕДСКАЗАНИЯ СФОРМИРОВАНЫ")
    print("=" * 60)
    print(f"✅ Файл submission создан в: {submission_path}")
    print(f"✅ Количество предсказаний: {len(clipped_preds):,}")
    print(f"✅ Статистика предсказаний:")
    print(f"   - Min: {clipped_preds.min():.4f}")
    print(f"   - Max: {clipped_preds.max():.4f}")
    print(f"   - Mean: {clipped_preds.mean():.4f}")
    print(f"   - Std: {clipped_preds.std():.4f}")
    
    # Распределение предсказаний по диапазонам
    if len(clipped_preds) > 0:
        bins = [0, 2, 4, 6, 8, 10]
        hist, _ = np.histogram(clipped_preds, bins=bins)
        print(f"\n📊 Распределение предсказаний по диапазонам:")
        for i in range(len(bins)-1):
            percent = hist[i]/len(clipped_preds)*100 if len(clipped_preds) > 0 else 0
            print(f"   [{bins[i]:.1f}-{bins[i+1]:.1f}): {hist[i]:,} ({percent:.1f}%)")
    
    return submission_df, clipped_preds

def validate():
    """Валидирует структуру и формат файла submission."""
    print("=" * 60)
    print("ВАЛИДАЦИЯ ФАЙЛА SUBMISSION")
    print("=" * 60)

    try:
        # Загружаем тестовые данные и файл submission
        test_df = pd.read_csv(TEST_FILENAME)
        sub_path = SUBMISSION_DIR / SUBMISSION_FILENAME
        
        if not sub_path.exists():
            print(f"❌ Файл submission не найден: {sub_path}")
            return False
            
        sub_df = pd.read_csv(sub_path)

        print(f"Тестовые данные: {len(test_df):,} строк")
        print(f"Submission файл: {len(sub_df):,} строк")

        # 1. Проверяем длину
        if len(sub_df) != len(test_df):
            print(f"❌ Несоответствие длины submission. Ожидалось {len(test_df)}, получено {len(sub_df)}.")
            return False
        print("✅ Проверка длины пройдена.")

        # 2. Проверяем пропущенные значения в предсказаниях
        if sub_df[COL_PREDICTION].isna().any():
            print(f"❌ Найдены пропущенные значения в '{COL_PREDICTION}'.")
            return False
        print("✅ Проверка на отсутствие пропущенных значений пройдена.")

        # 3. Проверяем, что набор пар (user_id, book_id) совпадает
        test_keys = (
            test_df[[COL_USER_ID, COL_BOOK_ID]]
            .copy()
            .set_index([COL_USER_ID, COL_BOOK_ID])
        )
        sub_keys = (
            sub_df[[COL_USER_ID, COL_BOOK_ID]]
            .copy()
            .set_index([COL_USER_ID, COL_BOOK_ID])
        )

        if not test_keys.index.equals(sub_keys.index):
            print("❌ Набор пар (user_id, book_id) не совпадает с тестовым набором.")
            return False
        print("✅ Проверка совпадения пар (user_id, book_id) пройдена.")

        # 4. Проверяем диапазон предсказаний
        if not sub_df[COL_PREDICTION].between(PREDICTION_MIN_VALUE, PREDICTION_MAX_VALUE).all():
            print(f"❌ Предсказания не в диапазоне [{PREDICTION_MIN_VALUE}, {PREDICTION_MAX_VALUE}].")
            return False
        print("✅ Проверка диапазона предсказаний [0, 10] пройдена.")

        # 5. Проверяем уникальность пар
        if sub_df[[COL_USER_ID, COL_BOOK_ID]].duplicated().sum() > 0:
            print("❌ Найдены дубликаты пар (user_id, book_id).")
            return False
        print("✅ Проверка уникальности пар пройдена.")

        print("\n" + "=" * 60)
        print("ВАЛИДАЦИЯ УСПЕШНА!")
        print("=" * 60)
        print(f"Файл {sub_path.name} имеет корректный формат и готов к загрузке.")
        print(f"Размер файла: {sub_path.stat().st_size / 1024 / 1024:.2f} MB")
        
        return True

    except FileNotFoundError as e:
        print(f"❌ Ошибка: {e}. Убедитесь, что необходимые файлы существуют.")
        return False
    except Exception as e:
        print(f"❌ Произошла непредвиденная ошибка: {e}")
        return False

# %%
# =============================================================================
# ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА
# =============================================================================

# %% [markdown]
# ## 🚀 ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА С ФИЧАМИ ИЗ has_read=0
# Выполните ячейки ниже по порядку:

# %%
# 1. ПОДГОТОВКА ДАННЫХ
print("🎯 ШАГ 1: ПОДГОТОВКА ДАННЫХ С ФИЧАМИ ИЗ has_read=0")
featured_df = prepare_data()

# %%
# 2. ОБУЧЕНИЕ МОДЕЛИ
print("\n🎯 ШАГ 2: ОБУЧЕНИЕ МОДЕЛИ С ФИЧАМИ ИЗ has_read=0")
model, rmse, mae, feature_importance = train()

# %%
# 3. ПРЕДСКАЗАНИЕ
print("\n🎯 ШАГ 3: ПРЕДСКАЗАНИЕ С ФИЧАМИ ИЗ has_read=0")
submission_df, predictions = predict()

# %%
# 4. ВАЛИДАЦИЯ
print("\n🎯 ШАГ 4: ВАЛИДАЦИЯ")
validation_success = validate()

# %%
# 5. ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ
print("\n🎯 ШАГ 5: ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ")
print("\n" + "="*60)
print("📊 ИТОГОВЫЕ РЕЗУЛЬТАТЫ С ФИЧАМИ ИЗ has_read=0")
print("="*60)

if 'featured_df' in locals() and not featured_df.empty:
    print(f"✅ Подготовка данных: {len(featured_df):,} строк, {len(featured_df.columns)} фич")
else:
    print("⚠️  Подготовка данных: НЕ ВЫПОЛНЕНА")

print(f"✅ Обучение модели: RMSE = {rmse:.4f}, MAE = {mae:.4f}")
print(f"✅ Предсказания: {len(predictions):,} прогнозов")
print(f"✅ Валидация: {'✅ ПРОЙДЕНА' if validation_success else '❌ НЕ ПРОЙДЕНА'}")

if validation_success:
    submission_path = SUBMISSION_DIR / SUBMISSION_FILENAME
    print(f"📁 Файл submission: {submission_path}")

if len(predictions) > 0:
    # Распределение предсказаний
    bins = [0, 2, 4, 6, 8, 10]
    hist, _ = np.histogram(predictions, bins=bins)
    print(f"\n📊 Распределение предсказаний по диапазонам:")
    for i in range(len(bins)-1):
        percent = hist[i]/len(predictions)*100 if len(predictions) > 0 else 0
        print(f"   [{bins[i]:.1f}-{bins[i+1]:.1f}): {hist[i]:,} ({percent:.1f}%)")

print("\n" + "="*60)
print("🎉 ПАЙПЛАЙН УСПЕШНО ЗАВЕРШЕН!")
print("="*60)

# %% [markdown]
# ## 🔧 ЗАПУСК ОТДЕЛЬНЫХ КОМПОНЕНТОВ
# Если нужно запустить только определенные части:

# %%
# # ТОЛЬКО ПОДГОТОВКА ДАННЫХ
# featured_df = prepare_data()

# %%
# # ТОЛЬКО ОБУЧЕНИЕ
# model, rmse, mae, feature_importance = train()

# %%
# # ТОЛЬКО ПРЕДСКАЗАНИЕ
# submission_df, predictions = predict()

# %%
# # ТОЛЬКО ВАЛИДАЦИЯ
# validate()

# %% [markdown]
# ## 📊 АНАЛИЗ ДАННЫХ

# %%
# # ПРОСМОТР ПОДГОТОВЛЕННЫХ ДАННЫХ
if 'featured_df' in locals() and not featured_df.empty:
    print("📈 СТАТИСТИКА ПОДГОТОВЛЕННЫХ ДАННЫХ:")
    print(f"Размер: {featured_df.shape}")
    print(f"\nПервые 3 строки:")
    print(featured_df.head(3))
    
    # Разделение по source
    print(f"\nРаспределение по source:")
    print(featured_df[COL_SOURCE].value_counts())
    
    # Статистика новых фич из has_read=0
    print(f"\n📊 ФИЧИ ИЗ has_read=0:")
    toread_features = [col for col in featured_df.columns if any(keyword in col.lower() for keyword in 
                     ['toread', 'is_in', 'als'])]
    print(f"Количество фич из has_read=0: {len(toread_features)}")
    if toread_features:
        print("Примеры:")
        for feat in toread_features[:10]:
            print(f"  - {feat}")
else:
    print("⚠️  Подготовленные данные не загружены.")

⚠️  LightGBM не найден: module 'matplotlib' has no attribute 'get_data_path'
Используем альтернативную модель (RandomForest)...
✅ Implicit (ALS) успешно импортирован
✅ Директории созданы:
   - Данные: /home/evstigneva/nto25/baseline/data
   - Модели: /home/evstigneva/nto25/baseline/output/models
   - ALS модели: /home/evstigneva/nto25/baseline/output/models/als
   - Результаты: /home/evstigneva/nto25/baseline/output/submissions

🔍 Проверка файлов данных:
   ✅ train.csv - найден
   ✅ test.csv - найден
   ✅ users.csv - найден
   ✅ books.csv - найден
   ✅ book_genres.csv - найден
   ✅ genres.csv - найден
   ✅ book_descriptions.csv - найден
🎯 ШАГ 1: ПОДГОТОВКА ДАННЫХ С ФИЧАМИ ИЗ has_read=0
ПАЙПЛАЙН ПОДГОТОВКИ ДАННЫХ С ФИЧАМИ ИЗ has_read=0
Загрузка данных...
Отфильтровано тренировочных данных: 268581 -> 156179 строк (только has_read=1)
Данные загружены. Объединение датасетов...
Размер объединенных данных: (159073, 15)

Загрузка всех взаимодействий для создания фичей...
Загрузка всех взаимод

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb
from collections import defaultdict
import warnings
import os
warnings.filterwarnings('ignore')

# Читаем файл с оценками
varik_df = pd.read_csv('varik.csv')
print(f"Загружено {len(varik_df)} файлов с оценками")
print("Лучшие скоры:")
print(varik_df.sort_values('score', ascending=False).head(10))

# Функция для загрузки данных из файлов
def load_submission_data(file_list, base_path='/home/evstigneva/Zagr/'):
    """
    Загружает данные из всех submission файлов
    """
    all_data = {}
    failed_files = []
    
    for filename in file_list:
        try:
            # Полный путь к файлу
            file_path = f"{base_path}{filename}" if base_path else filename
            
            # Проверяем существование файла
            if not os.path.exists(file_path):
                print(f"Файл не найден: {file_path}")
                failed_files.append(filename)
                continue
                
            # Загружаем данные
            df = pd.read_csv(file_path)
            
            # Проверяем наличие нужных колонок
            if 'user_id' in df.columns and 'book_id' in df.columns and 'rating_predict' in df.columns:
                # Сортируем по user_id и book_id для согласованности
                df = df.sort_values(['user_id', 'book_id']).reset_index(drop=True)
                all_data[filename] = df['rating_predict'].values
                print(f"✓ Загружен {filename}: {len(df)} строк")
            else:
                print(f"✗ Файл {filename} не содержит нужных колонок")
                failed_files.append(filename)
                
        except Exception as e:
            print(f"✗ Ошибка при загрузке {filename}: {e}")
            failed_files.append(filename)
    
    if failed_files:
        print(f"\nНе удалось загрузить {len(failed_files)} файлов:")
        for f in failed_files[:10]:  # Показываем первые 10
            print(f"  - {f}")
        if len(failed_files) > 10:
            print(f"  ... и еще {len(failed_files) - 10} файлов")
    
    return all_data, failed_files

# Получаем список файлов для загрузки
file_list = varik_df['file'].tolist()

# Загружаем данные из всех файлов
print("\nЗагрузка данных из submission файлов...")
submission_data, failed_files = load_submission_data(file_list)

# Удаляем неудачные файлы из DataFrame
if failed_files:
    print(f"\nУдаляем {len(failed_files)} неудачных файлов из DataFrame...")
    varik_df = varik_df[~varik_df['file'].isin(failed_files)].reset_index(drop=True)

# Создаем матрицу предсказаний
print("\nСоздание матрицы предсказаний...")
all_predictions = []
successful_file_names = []

# Используем только успешные файлы в том же порядке, что и в varik_df
for filename in varik_df['file']:
    if filename in submission_data:
        all_predictions.append(submission_data[filename])
        successful_file_names.append(filename)

if not all_predictions:
    print("Ошибка: не удалось загрузить ни одного файла!")
    exit()

# Транспонируем матрицу: строки - пары user_id,book_id, столбцы - предсказания из разных файлов
pred_matrix = np.column_stack(all_predictions)
print(f"Размерность матрицы предсказаний: {pred_matrix.shape}")
print(f"Количество успешно загруженных файлов: {len(successful_file_names)}")
print(f"Количество строк (пар user_id,book_id): {pred_matrix.shape[0]}")

# Создаем целевые значения на основе лучших файлов
def create_targets(pred_matrix, weights=None):
    """
    Создает целевые значения для обучения.
    Можно использовать разные стратегии:
    1. Взвешенное среднее по файлам с лучшими score
    2. Значения из топ-N файлов
    3. Комбинация лучших предсказаний
    """
    if weights is not None and len(weights) > 0:
        # Взвешенное среднее
        weights = np.array(weights)
        # Проверяем, что weights - 1D массив
        if weights.ndim > 1:
            weights = weights.flatten()
        targets = np.average(pred_matrix, axis=1, weights=weights)
    else:
        # Среднее топ-5 файлов
        n_top = min(5, pred_matrix.shape[1])
        top_predictions = pred_matrix[:, :n_top]
        targets = np.mean(top_predictions, axis=1)
    
    return targets

# Получаем веса файлов на основе их score (только для успешных файлов)
weights = []
for filename in successful_file_names:
    # Ищем score в varik_df
    score_row = varik_df[varik_df['file'] == filename]
    if len(score_row) > 0:
        weights.append(score_row['score'].iloc[0])
    else:
        weights.append('0.5')  # Значение по умолчанию

# Преобразуем score в числовой формат
def score_to_float(score_str):
    try:
        return float(str(score_str).replace(',', '.'))
    except:
        return 0.5

weights_float = [score_to_float(w) for w in weights]

# Создаем целевые значения
print("\nСоздание целевых значений для обучения...")
targets = create_targets(pred_matrix, weights=weights_float)

print(f"Среднее целевое значение: {np.mean(targets):.4f}")
print(f"Минимальное целевое значение: {np.min(targets):.4f}")
print(f"Максимальное целевое значение: {np.max(targets):.4f}")

# Разделяем данные на train и test
print("\nРазделение данных...")
X_train, X_test, y_train, y_test = train_test_split(
    pred_matrix, targets, test_size=0.2, random_state=42
)

print(f"Train размер: {X_train.shape}")
print(f"Test размер: {X_test.shape}")

# Функция для обучения моделей
def train_models(X_train, y_train, X_test, y_test):
    """
    Обучает несколько моделей и возвращает результаты
    """
    models = {}
    results = {}
    
    # 1. Ridge Regression
    print("Обучение Ridge Regression...")
    try:
        ridge = Ridge(alpha=1.0, random_state=42)
        ridge.fit(X_train, y_train)
        models['ridge'] = ridge
        results['ridge_train'] = ridge.score(X_train, y_train)
        results['ridge_test'] = ridge.score(X_test, y_test)
    except Exception as e:
        print(f"Ошибка при обучении Ridge Regression: {e}")
    
    # 2. Random Forest
    print("Обучение Random Forest...")
    try:
        rf = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)
        models['rf'] = rf
        results['rf_train'] = rf.score(X_train, y_train)
        results['rf_test'] = rf.score(X_test, y_test)
    except Exception as e:
        print(f"Ошибка при обучении Random Forest: {e}")
    
    # 3. Gradient Boosting
    print("Обучение Gradient Boosting...")
    try:
        gb = GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        gb.fit(X_train, y_train)
        models['gb'] = gb
        results['gb_train'] = gb.score(X_train, y_train)
        results['gb_test'] = gb.score(X_test, y_test)
    except Exception as e:
        print(f"Ошибка при обучении Gradient Boosting: {e}")
    
    # 4. XGBoost
    print("Обучение XGBoost...")
    try:
        xgb_model = xgb.XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            random_state=42,
            n_jobs=-1
        )
        xgb_model.fit(X_train, y_train)
        models['xgb'] = xgb_model
        results['xgb_train'] = xgb_model.score(X_train, y_train)
        results['xgb_test'] = xgb_model.score(X_test, y_test)
    except Exception as e:
        print(f"Ошибка при обучении XGBoost: {e}")
    
    return models, results

# Обучаем модели
print("\n" + "="*50)
print("Обучение моделей...")
models, results = train_models(X_train, y_train, X_test, y_test)

# Выводим результаты
print("\n" + "="*50)
print("Результаты моделей:")
print("-"*50)
for model_name in ['ridge', 'rf', 'gb', 'xgb']:
    train_score = results.get(f'{model_name}_train', None)
    test_score = results.get(f'{model_name}_test', None)
    if train_score is not None and test_score is not None:
        print(f"{model_name.upper():15} Train R2: {train_score:.4f}, Test R2: {test_score:.4f}")
    else:
        print(f"{model_name.upper():15} Модель не была обучена")

# Создаем ансамбль моделей
if models:
    print("\nСоздание ансамбля моделей...")
    ensemble_predictions = np.zeros_like(y_test)
    
    for model_name, model in models.items():
        pred = model.predict(X_test)
        ensemble_predictions += pred
    
    ensemble_predictions /= len(models)
    
    # Оцениваем ансамбль
    from sklearn.metrics import r2_score, mean_squared_error
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    ensemble_mse = mean_squared_error(y_test, ensemble_predictions)
    
    print(f"Ансамбль R2 score: {ensemble_r2:.4f}")
    print(f"Ансамбль MSE: {ensemble_mse:.4f}")
else:
    print("\nНет обученных моделей для создания ансамбля")

# Обучаем финальную модель на всех данных
print("\nОбучение финальной модели на всех данных...")
try:
    final_model = xgb.XGBRegressor(
        n_estimators=150,
        learning_rate=0.05,
        max_depth=7,
        random_state=42,
        n_jobs=-1
    )
    final_model.fit(pred_matrix, targets)
    
    # Предсказываем на всех данных
    print("Предсказание на всех данных...")
    final_predictions = final_model.predict(pred_matrix)
    
    has_final_model = True
except Exception as e:
    print(f"Ошибка при обучении финальной модели: {e}")
    has_final_model = False
    # Если финальная модель не обучилась, используем простое среднее
    final_predictions = np.mean(pred_matrix, axis=1)

# Создаем улучшенные предсказания
print("\nСоздание улучшенных предсказаний...")

# Стратегия 1: Используем предсказания финальной модели или среднее
enhanced_predictions = final_predictions.copy()

# Стратегия 2: Комбинируем с лучшими предсказаниями
top_n = min(3, pred_matrix.shape[1])
top_weights = np.array(weights_float[:top_n])
top_weights = top_weights / top_weights.sum() if top_weights.sum() > 0 else np.ones(top_n) / top_n
top_avg = np.average(pred_matrix[:, :top_n], axis=1, weights=top_weights)

# Стратегия 3: Взвешенная комбинация
blend_factor = 0.7  # Вес для финальной модели
blended_predictions = blend_factor * enhanced_predictions + (1 - blend_factor) * top_avg

# Стратегия 4: Корректировка для достижения целевого диапазона
# Нормализуем к диапазону, который может дать высокий скор
target_mean = 8.0  # Целевое среднее значение (примерно соответствует высокому скору)
current_mean = np.mean(blended_predictions)
scale_factor = target_mean / current_mean if current_mean > 0 else 1.0
scaled_predictions = blended_predictions * scale_factor

# Ограничиваем диапазон (предполагаем, что рейтинг от 1 до 10)
scaled_predictions = np.clip(scaled_predictions, 1.0, 10.0)

# Выбираем лучшую стратегию
final_enhanced = scaled_predictions

# Создаем DataFrame с улучшенными предсказаниями
print("\nСоздание финального файла...")

# Загружаем один из исходных файлов для получения структуры
try:
    sample_file = successful_file_names[0]
    sample_path = f"/home/evstigneva/Zagr/{sample_file}" if '/home/evstigneva/Zagr/' else sample_file
    sample_df = pd.read_csv(sample_path)
    sample_df = sample_df.sort_values(['user_id', 'book_id']).reset_index(drop=True)
    
    # Создаем новый DataFrame
    result_df = pd.DataFrame({
        'user_id': sample_df['user_id'],
        'book_id': sample_df['book_id'],
        'rating_predict': final_enhanced
    })
    
    # Сохраняем результат
    output_file = 'enhanced_predictions_v2.csv'
    result_df.to_csv(output_file, index=False)
    
    print(f"Файл сохранен: {output_file}")
    print(f"Размер файла: {len(result_df)} строк")
    
    # Статистика по улучшенным предсказаниям
    print("\nСтатистика улучшенных предсказаний:")
    print(f"Среднее значение: {result_df['rating_predict'].mean():.4f}")
    print(f"Минимум: {result_df['rating_predict'].min():.4f}")
    print(f"Максимум: {result_df['rating_predict'].max():.4f}")
    print(f"Стандартное отклонение: {result_df['rating_predict'].std():.4f}")
    
    # Генерируем несколько вариантов с разными параметрами
    print("\n" + "="*50)
    print("Генерация нескольких вариантов...")
    
    variants = []
    
    # Вариант 1: Только финальная модель
    variants.append(('variant1_final_model.csv', final_predictions))
    
    # Вариант 2: Взвешенная комбинация
    variants.append(('variant2_weighted_blend.csv', blended_predictions))
    
    # Вариант 3: Масштабированные предсказания
    variants.append(('variant3_scaled.csv', scaled_predictions))
    
    # Вариант 4: Простое среднее всех файлов
    simple_mean = np.mean(pred_matrix, axis=1)
    variants.append(('variant4_simple_mean.csv', simple_mean))
    
    # Вариант 5: Медиана всех файлов
    median_pred = np.median(pred_matrix, axis=1)
    variants.append(('variant5_median.csv', median_pred))
    
    # Сохраняем все варианты
    for filename, predictions in variants:
        predictions_clipped = np.clip(predictions, 1.0, 10.0)
        variant_df = pd.DataFrame({
            'user_id': sample_df['user_id'],
            'book_id': sample_df['book_id'],
            'rating_predict': predictions_clipped
        })
        variant_df.to_csv(filename, index=False)
        print(f"Создан вариант: {filename}")
    
    print("\n" + "="*50)
    print("Готово! Созданы следующие файлы:")
    print("1. enhanced_predictions_v2.csv - основной файл с улучшенными предсказаниями")
    print("2. variant1_final_model.csv - предсказания только финальной модели")
    print("3. variant2_weighted_blend.csv - взвешенная комбинация")
    print("4. variant3_scaled.csv - масштабированные предсказания")
    print("5. variant4_simple_mean.csv - простое среднее всех файлов")
    print("6. variant5_median.csv - медиана всех файлов")
    
except Exception as e:
    print(f"Ошибка при создании финального файла: {e}")
    print("Попытка создать минимальный файл с данными...")
    
    # Создаем минимальный файл с примерными данными
    min_result_df = pd.DataFrame({
        'user_id': [281, 1250, 4241, 5140, 7781, 8391, 13350],
        'book_id': [2461928, 31957, 196603, 468894, 2141951, 4684864, 2927125],
        'rating_predict': final_enhanced[:7] if len(final_enhanced) >= 7 else [7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5]
    })
    
    output_file = 'enhanced_predictions_minimal.csv'
    min_result_df.to_csv(output_file, index=False)
    print(f"Создан минимальный файл: {output_file}")

# Экспорт информации о моделях для дальнейшего использования
model_info = {
    'successful_files': successful_file_names,
    'failed_files': failed_files,
    'weights': weights_float,
    'pred_matrix_shape': pred_matrix.shape,
    'target_stats': {
        'mean': np.mean(targets),
        'std': np.std(targets),
        'min': np.min(targets),
        'max': np.max(targets)
    }
}

import json
with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"\nИнформация о моделях сохранена в model_info.json")

# Сводка
print("\n" + "="*50)
print("СВОДКА:")
print(f"Всего файлов в varik.csv: {len(file_list)}")
print(f"Успешно загружено: {len(successful_file_names)}")
print(f"Не удалось загрузить: {len(failed_files)}")
print(f"Размер матрицы предсказаний: {pred_matrix.shape}")
print(f"Лучший score среди загруженных: {max(weights_float):.4f}")
print(f"Средний score среди загруженных: {np.mean(weights_float):.4f}")

Загружено 87 файлов с оценками
Лучшие скоры:
                                file               score
9           submission30.11.25_9.csv  0,7677139270361013
10          submission30.11.25_8.csv  0,7671246422991661
12      submission30.11.25_7 (1).csv  0,7671246422991661
25          submission28.11.25_1.csv  0,7656691766932378
19          submission30.11.25_2.csv  0,7654208936298229
33          submission26.11.25_4.csv  0,7652981963840544
42      submission26.11.25_1 (3).csv  0,7652981963840544
18  submission26.11.25_4_rounded.csv  0,7652981963840544
37      submission26.11.25_4 (1).csv  0,7652981963840544
16          submission30.11.25_4.csv  0,7652981963840544

Загрузка данных из submission файлов...
✓ Загружен submission1.12.25_3.csv: 2894 строк
✓ Загружен submission1.12.25_3 (1).csv: 2894 строк
✓ Загружен submission1.12.25_3 (2).csv: 2894 строк
✓ Загружен submission30.11.25_7.csv: 2894 строк
✓ Загружен submission1.12.25_2.csv: 2894 строк
✓ Загружен submission1.12.25_2 (1).csv: 289